In [1]:
!pip install datasets
!pip install -U sentence-transformers
!sudo apt-get install git-lfs

  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120748 sha256=41d54e9d369080a89fc4fc2cf5e14178a88b1b7bfca191a5a414412b6003b338
  Stored in directory: c:\users\javier.sanz\appdata\local\pip\cache\wheels\0c\b6\fb\2289a932c365293ad865fc1fe9d2db694d5584241c6d670874
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0


"sudo" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
!pip install datasets

In [3]:
import json
import math
import pandas as pd
from datasets import load_dataset,Dataset,DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, r2_score, roc_curve
from sentence_transformers import SentenceTransformer, InputExample, losses, util, evaluation, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
from torch import nn

In [4]:
import datasets
import sklearn
import sentence_transformers
import torch

In [ ]:
!git config --global user.email "jfarray@gmail.com"
!git config --global user.name "jfarray"
!huggingface-cli login 


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Username: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/huggingface_hub/commands/user.py", line 166, in run
    token = getpass("Token

In [7]:
def ConvertJsonToList(fileName, split):
  subject1_fileDataset = load_dataset("json", data_files=fileName, split="train")
  samples = []

  for i in range (0,len(subject1_fileDataset[split][0])): #len(subject1)
    mark = subject1_fileDataset[split][0][i]['mark']
    responseStudent = subject1_fileDataset[split][0][i]['responseStudent']
    responseTeacher = subject1_fileDataset[split][0][i]['responseTeacher']
    
    ie = InputExample(texts=[responseTeacher, responseStudent], label=mark)
    samples.append(ie)

  return samples    

In [8]:
#Creating a list with the necesarry fields
data_train = ConvertJsonToList("Downloads/JoinedSubjects2.json", "train")
data_test = ConvertJsonToList("Downloads/JoinedSubjects2.json", "test")
data_valid = ConvertJsonToList("Downloads/JoinedSubjects2.json", "valid")

Using custom data configuration default-5220c3d69df0244d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to C:\Users\javier.sanz\.cache\huggingface\datasets\json\default-5220c3d69df0244d\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


Using custom data configuration default-5220c3d69df0244d
Reusing dataset json (C:\Users\javier.sanz\.cache\huggingface\datasets\json\default-5220c3d69df0244d\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)
Using custom data configuration default-5220c3d69df0244d
Reusing dataset json (C:\Users\javier.sanz\.cache\huggingface\datasets\json\default-5220c3d69df0244d\0.0.0\ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


In [9]:
def CreateModelEvaluationData():
  sentences1 = []
  sentences2 = []
  scores = []

  for i in range (0,len(data_valid)):
    sentences1.append(data_valid[i].texts[0])
    sentences2.append(data_valid[i].texts[1])
    scores.append(data_valid[i].label)

  evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)
  return evaluator

In [10]:
def TrainModel(checkpoint, evaluator, epochs, fromScratch):
  batch_size = int(len(data_train) * 0.1)
  #Create the model from checkpoint
  if (not fromScratch):
    model = SentenceTransformer(checkpoint)
  else:
    word_embedding_model = models.Transformer(checkpoint, max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

  train_dataloader = DataLoader(data_train, shuffle=True, batch_size=batch_size)
  train_loss = losses.CosineSimilarityLoss(model)

  #Fit the model
  local_model_path = 'Downloads/Model_' + checkpoint + '/' + str(epochs) + '_Epochs'
  warmup_steps = math.ceil(len(train_dataloader) * epochs * 0.1) #10% of train data for warm-up
  evaluation_steps = int(len(train_dataloader)*0.1)
  print(len(train_dataloader),warmup_steps,evaluation_steps)
  model.fit(train_objectives=[(train_dataloader, train_loss)]
            , epochs=epochs
            , warmup_steps=warmup_steps
            , evaluator=evaluator
            , evaluation_steps=evaluation_steps
            ,output_path=local_model_path
            ,save_best_model=True)
  
  
  model.save("Downloads/models/"+checkpoint+ str(epochs))

In [11]:
modelsToTrain = [
                  {"checkPoint": "distiluse-base-multilingual-cased-v1", "fromScratch": False},
                  {"checkPoint": "paraphrase-multilingual-MiniLM-L12-v2", "fromScratch": False},
                  {"checkPoint": "paraphrase-multilingual-mpnet-base-v2", "fromScratch": False},
                  {"checkPoint": "all-distilroberta-v1", "fromScratch": False},
                  {"checkPoint": "bert-base-multilingual-uncased", "fromScratch": True},
                  {"checkPoint": "dccuchile/bert-base-spanish-wwm-uncased", "fromScratch": True}
              ]

#epochsToTest = [1,5,10,30,50,100]
epochsToTest = [1]

#Get evaluator
evaluator = CreateModelEvaluationData()

#Train the models
for model in modelsToTrain:
  for epochs in epochsToTest:
    TrainModel(model["checkPoint"], evaluator, epochs, model["fromScratch"])

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

11 2 1


C:\Users\javier.sanz\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
import os
from google.colab import files
for i in os.listdir('/content/') :  
 #print(i)
  files.download('/content/'+i)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

from google.colab import files
import os

for checkpoint in modelsToTest:
  dir_to_zip = '/content/Model_' + checkpoint 
  output_filename = '/content/'+ checkpoint + '.zip' 
  delete_dir_after_download = "No"  

  os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

  if delete_dir_after_download == "Yes":
      os.system( "rm -r {}".format( dir_to_zip ) )

  #files.download( output_filename )